# height 결측값 처리
* 1층 건물의 경우 1층 건물들의 평균 값으로 설정
* 2층 이상의 건물의 경우 층당 3m로 설정하여 계산

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Scored 광주건물.csv", encoding="ansi")
df.head()

,location,lat,lon,structure,area,height,ground,underground,date,seismic,Score
0,광주광역시 광산구 고룡동 100-1번지,35.204703,126.777928,일반목구조,124.13,0.0,1,0,1974,0,1.976301
1,광주광역시 광산구 고룡동 1000번지,35.202519,126.777766,일반철골구조,974.70,10.2,2,0,2016,1,-0.697155
2,광주광역시 광산구 고룡동 1001번지,35.190152,126.773402,일반철골구조,958.17,11.2,2,0,2016,1,-0.697155
3,광주광역시 광산구 고룡동 1003번지,35.137946,126.791957,일반철골구조,887.92,7.4,2,0,2016,0,1.339727
4,광주광역시 광산구 고룡동 1005번지,35.190365,126.774364,일반철골구조,331.00,8.2,1,0,2017,1,-0.879477


In [3]:
mean = df[df.height!=0][df.ground==1]["height"].mean()
mean

<ipython-input-3-77aad20e73de>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mean = df[df.height!=0][df.ground==1]["height"].mean()


5.9057131246757875

In [4]:
def height_cal(height, ground):
    if height != 0:
        return height
    else:
        if ground == 1:
            return mean
        else:
            return 3 * ground

In [5]:
df["height"] = df.apply(lambda x : height_cal(x["height"], x["ground"]), axis=1)
df

,location,lat,lon,structure,area,height,ground,underground,date,seismic,Score
0,광주광역시 광산구 고룡동 100-1번지,35.204703,126.777928,일반목구조,124.13,5.905713,1,0,1974,0,1.976301
1,광주광역시 광산구 고룡동 1000번지,35.202519,126.777766,일반철골구조,974.70,10.200000,2,0,2016,1,-0.697155
2,광주광역시 광산구 고룡동 1001번지,35.190152,126.773402,일반철골구조,958.17,11.200000,2,0,2016,1,-0.697155
3,광주광역시 광산구 고룡동 1003번지,35.137946,126.791957,일반철골구조,887.92,7.400000,2,0,2016,0,1.339727
4,광주광역시 광산구 고룡동 1005번지,35.190365,126.774364,일반철골구조,331.00,8.200000,1,0,2017,1,-0.879477
...,...,...,...,...,...,...,...,...,...,...,...
122592,광주광역시 북구 오룡동 산 23-1번지,35.214971,126.844826,일반철골구조,19.83,5.905713,1,0,1960,0,2.136058
122593,광주광역시 북구 오룡동 산 24-3번지,35.228358,126.847842,일반철골구조,40.00,5.905713,1,0,1960,0,2.136058
122594,광주광역시 북구 오룡동 산 67번지,35.233707,126.842511,일반철골구조,29.23,5.905713,1,0,1960,0,2.136058
122595,광주광역시 북구 오룡동 산 70번지,35.233860,126.843889,일반철골구조,58.55,5.905713,1,0,1960,0,2.136058


# 위험 단계에 따른 범위 계산
* 주의 < 위험 < 붕괴 의 순서로 위험도가 올라가며 이에 따라 피해 범위도 증가함

In [6]:
def cal_damage1(area, height):
    return np.sqrt(area)

def cal_damage2(area, height):
    return np.sqrt(area) + (height * np.tan(np.pi/60))

def cal_damage3(area, height):
    return np.sqrt(area) + (height * np.tan(np.pi/30))

In [7]:
df["damage_level1"] = df.apply(lambda x : cal_damage1(x["area"], x["height"]), axis = 1 )
df["damage_level2"] = df.apply(lambda x : cal_damage2(x["area"], x["height"]), axis = 1 )
df["damage_level3"] = df.apply(lambda x : cal_damage3(x["area"], x["height"]), axis = 1 )
df

,location,lat,lon,structure,area,height,ground,underground,date,seismic,Score,damage_level1,damage_level2,damage_level3
0,광주광역시 광산구 고룡동 100-1번지,35.204703,126.777928,일반목구조,124.13,5.905713,1,0,1974,0,1.976301,11.141364,11.450870,11.762080
1,광주광역시 광산구 고룡동 1000번지,35.202519,126.777766,일반철골구조,974.70,10.200000,2,0,2016,1,-0.697155,31.220186,31.754745,32.292249
2,광주광역시 광산구 고룡동 1001번지,35.190152,126.773402,일반철골구조,958.17,11.200000,2,0,2016,1,-0.697155,30.954321,31.541288,32.131489
3,광주광역시 광산구 고룡동 1003번지,35.137946,126.791957,일반철골구조,887.92,7.400000,2,0,2016,0,1.339727,29.797987,30.185804,30.575758
4,광주광역시 광산구 고룡동 1005번지,35.190365,126.774364,일반철골구조,331.00,8.200000,1,0,2017,1,-0.879477,18.193405,18.623149,19.055260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122592,광주광역시 북구 오룡동 산 23-1번지,35.214971,126.844826,일반철골구조,19.83,5.905713,1,0,1960,0,2.136058,4.453089,4.762594,5.073804
122593,광주광역시 북구 오룡동 산 24-3번지,35.228358,126.847842,일반철골구조,40.00,5.905713,1,0,1960,0,2.136058,6.324555,6.634061,6.945271
122594,광주광역시 북구 오룡동 산 67번지,35.233707,126.842511,일반철골구조,29.23,5.905713,1,0,1960,0,2.136058,5.406478,5.715983,6.027193
122595,광주광역시 북구 오룡동 산 70번지,35.233860,126.843889,일반철골구조,58.55,5.905713,1,0,1960,0,2.136058,7.651797,7.961302,8.272513


In [8]:
df.to_csv("DamageAreaPredict 광주건물.csv", sep=",", na_rep="NaN", encoding="ansi", index = False)